<a href="https://colab.research.google.com/github/JunSeokCheon/Natural-Language-based-Climate-Technology-Classification/blob/master/Bert(Mecab)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
ls

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [ ]:
pip install konlpy

In [ ]:
pip install transformers==3.3

In [ ]:
from konlpy.tag import Mecab

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import *
#from transformers import BertModel, TFBertModel

In [ ]:
cd /content/drive/MyDrive/daicon/data

In [ ]:
train=pd.read_csv('preprocessing_train.csv')
test=pd.read_csv('preprocessing_test.csv')
sample_submission=pd.read_csv('sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
print(f'train.shape:{train.shape}')
print(f'test.shape:{test.shape}')
print(f'train label 개수: {train.label.nunique()}')

In [ ]:
#이번 베이스라인에서는 과제명 뿐만 아니라 요약문_연구내용도 모델에 학습시켜보겠습니다.
train=train[['사업명', '사업_부처명', '과제명', '요약문_연구목표', '요약문_한글키워드', '요약문_영문키워드', '요약문_연구내용', 'label']]
test=test[['사업명', '사업_부처명', '과제명', '요약문_연구목표', '요약문_한글키워드', '요약문_영문키워드', '요약문_연구내용']]
train[['사업명', '사업_부처명', '과제명', '요약문_연구목표', '요약문_한글키워드', '요약문_영문키워드', '요약문_연구내용', 'label']].fillna('NAN', inplace=True)
test[['사업명', '사업_부처명', '과제명', '요약문_연구목표', '요약문_한글키워드', '요약문_영문키워드', '요약문_연구내용']].fillna('NAN', inplace=True)

In [ ]:
# train['data']=train['사업명']+ '\n' + train['사업_부처명'] + '\n' + train['과제명'] + '\n' + train['요약문_한글키워드'] + '\n' + train['요약문_영문키워드']
train['data']= train['과제명'] + '\n' + train['요약문_한글키워드']
# test['data']=test['사업명']+ '\n' + test['사업_부처명'] + '\n' + test['과제명'] + '\n' + test['요약문_한글키워드'] + '\n' + test['요약문_영문키워드']
test['data']= test['과제명'] + '\n' + test['요약문_한글키워드']

In [ ]:
train = train.fillna('')
test = test.fillna('')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
MAX_LEN= 256

In [ ]:
pip install sentencepiece

In [ ]:
from transformers import ElectraTokenizer

In [ ]:
from transformers import *
tokenizer= ElectraTokenizer.from_pretrained("/content/drive/MyDrive/daicon/koelectra-base-v3", do_lower_case=False)

def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict=tokenizer.encode_plus(
    text = sent, 
    add_special_tokens=True, 
    max_length=MAX_LEN, 
    pad_to_max_length=True, 
    return_attention_mask=True,
    truncation = True)
    
    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

In [ ]:
input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []
mecab = Mecab()

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ ]", " ", sent)
    word_text=mecab.morphs(sent_clean)
    word_review=[token for token in word_text if not len(token)==1]
    word_review = "\n".join(str(w) for w in word_review)
    return word_review

for train_sent, train_label in zip(train['data'], train['label']):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), MAX_LEN=MAX_LEN)
        # input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAX_LEN=MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
        train_data_labels.append(train_label)
        
    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
train_labels=np.asarray(train_data_labels, dtype=np.int32)

In [ ]:
print(train_input_ids[1])
print(train_attention_masks[1])
print(train_token_type_ids[1])
print(tokenizer.decode(train_input_ids[1]))

In [ ]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits


cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased', dir_path='bert_ckpt', num_class=46)

In [ ]:
# 학습 준비하기
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_name = "tf2_bert_classifier"

# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_loss', min_delta=0.0001,patience=3)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

checkpoint_path = os.path.join(model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

# 학습과 eval 시작
model_history = cls_model.fit(train_inputs, train_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

In [ ]:
model_history

In [ ]:
input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

for test_sent in test['data']:
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(test_sent), MAX_LEN=MAX_LEN)
        # input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAX_LEN=MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
       
    except Exception as e:
        print(e)
        print(test_sent)
        pass
    
test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)

In [ ]:
results = cls_model.predict(test_inputs)
results=tf.argmax(results, axis=1)

In [ ]:
results

In [ ]:
sample_submission['label']=results

In [ ]:
sample_submission.shape

In [ ]:
cd /content/drive/MyDrive/daicon/

/content/drive/MyDrive/daicon


In [ ]:
sample_submission.to_csv('submission/[과제명&요약문_한글키워드],  koelectra vocab, maxlen256, split0.1, mecab, 10epoch - 21.8.15.csv', index=None)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])